# PySpark Version of 4_2_Example_Data_Bank

This notebook is an automated best-effort translation from pandas to PySpark.
Some pandas-specific operations (corr matrix, complex plotting, chained indexing) were adapted or annotated with TODOs.


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

spark = SparkSession.builder.appName("BankExample").getOrCreate()

# Helper: null counts per column
def null_counts(df):
    exprs = [count(when(col(c).isNull(), c)).alias(c) for c in df.columns]
    return df.agg(*exprs)

# Helper: safe rename using select exprs
def rename_cols(df, mapping: dict):
    return df.select([col(c).alias(mapping.get(c,c)) for c in df.columns])


In [ ]:
# load libraries
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

spark = SparkSession.builder.appName("BankExample").getOrCreate()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [ ]:
!curl -L -o dataset_banco.csv "https://drive.google.com/uc?export=download&id=1QF2Bc5u7pC13LHXL0rC8FiwGUBVL_8Uh"
#https://drive.google.com/file/d/177oC4Pg-2Pzd_t5hcpWECkXAgBcO_CBM/view?usp=sharing

In [ ]:
!wget --no-cache -O dataset_banco1 -q "https://drive.google.com/uc?export=download&id=1QF2Bc5u7pC13LHXL0rC8FiwGUBVL_8Uh"

In [ ]:
#Cargamos los CSV con los Datos
clientes_Banco = spark.read.option("header", True).option("inferSchema", True).csv("/content/dataset_banco.csv")
clientes_Banco

In [ ]:
#Listado de Columnas Dataset
clientes_Banco.columns

In [ ]:
# Breve descripcion del dataset

# Mostramos el numero de filas y columnas
print(({"rows": clientes_Banco.count(), "cols": len(clientes_Banco.columns)}))

# Informacion Basica del Dataset
print('\n')
clientes_Banco.printSchema()

In [ ]:
#Verificar si hay valores nulos, isnull() nos permite detectar datos nulos,
# TODO: manual null count translation

In [ ]:
##Verificar si hay valores nulos, si lo prefieres puedes usar notnull() en vez de isnull():
clientes_Banco.notnull().sum()

In [ ]:
#dropna: Eliminar valores nulos, esto solo se recomienda cuando hay muchos datos

# Dado que tenemos entre 1 y dos valores nulos por columna podemos eliminarlos
print(clientes_Banco.isnull())
print('\n')
clientes_Banco.dropna(inplace=True)
print('\n')
clientes_Banco.printSchema()


In [ ]:
clientes_Banco.columns

In [ ]:
# Para obtener el resumen estadístico de las categoricas:
clientes_Banco.describe(include = 'object')

In [ ]:
# Conteo de los niveles en las columnas categóricas
cont_cat = ['job', 'marital', 'education', 'default','housing',
            'loan', 'contact', 'month', 'balance', 'poutcome', 'y']

for col in cont_cat:
  print(f'Columna {col}: {clientes_Banco[col].nunique()} niveles')

In [ ]:
clientes_Banco.describe()

In [ ]:
print(f'Tamaño del dataset antes de eliminar las filas repetidas es: {({"rows": clientes_Banco.count(), "cols": len(clientes_Banco.columns)})}')
clientes_Banco.drop_duplicates(inplace=True)
print(f'Tamaño del dataset después de eliminar las filas repetidas es: {({"rows": clientes_Banco.count(), "cols": len(clientes_Banco.columns)})}')

In [ ]:
clientes_Banco.printSchema()

In [ ]:
# Para cambiar el tipo puede usar cualquiera de los siguientes comandos

# df = df.withColumn("dataframe_column", df["dataframe_column"].astype(int))
# df = df.withColumn("dataframe_column", df["dataframe_column"].apply(int))
# df = df.withColumn("dataframe_column", df["dataframe_column"].map(int))
# df=df.astype({'dataframe_column': 'string', 'dataframe_column':'float64',
#              'dataframe_column': 'datetime64[ns]', 'dataframe_column': 'int'})

In [ ]:
# Generar gráficas individuales pues las variables numéricas
# están en rangos diferentes
col_num = ['age', 'balance', 'day', 'duration', 'campaign',
            'pdays', 'previous']


fig, ax = plt.subplots(nrows=7, ncols=1, figsize=(8,30))
fig.subplots_adjust(hspace=0.5)

for i, col in enumerate(col_num):
    sns.boxplot(x=col, data=clientes_Banco, ax=ax[i])
    ax[i].set_title(col)

In [ ]:
# Mostrar tamaño inicial del dataset
print("="*60)
print(f"📊 Tamaño del conjunto antes de corrección: {({"rows": clientes_Banco.count(), "cols": len(clientes_Banco.columns)})}")
print("="*60)

# Contar registros con edades mayores a 100
edad_incorrecta = clientes_Banco['age'][clientes_Banco['age'] > 100].count()
print(f"🔍 Registros con edad > 100 años: {edad_incorrecta}")

# Calcular medidas estadísticas
promedio = clientes_Banco['age'].mean()
mediana = clientes_Banco['age'].median()
moda = clientes_Banco['age'].mode()[0]

print("\n📈 Estadísticas descriptivas de la edad:")
print(f"   ➤ Promedio : {promedio:.2f}")
print(f"   ➤ Mediana  : {mediana}")
print(f"   ➤ Moda     : {moda}")

# Mostrar registros problemáticos (si existen)
if edad_incorrecta > 0:
    print("\n⚠️ Clientes con edad superior a 100 años:")
    print(clientes_Banco.loc[clientes_Banco['age'] > 100])
else:
    print("\n✅ No se encontraron registros con edad superior a 100 años.")

# Corregir valores mayores a 100 reemplazándolos por la mediana
clientes_Banco.loc[clientes_Banco['age'] > 100, 'age'] = mediana

# Confirmar la corrección
print("\n🔧 Corrección aplicada: valores > 100 reemplazados por la mediana.\n")

# Verificar nuevamente si quedan registros mayores a 100
restantes = clientes_Banco.loc[clientes_Banco['age'] > 100]
if len(restantes) == 0:
    print("✅ Todos los registros fueron corregidos correctamente.")
else:
    print("⚠️ Aún hay registros con edad > 100:")
    print(restantes)

# Tamaño final del dataset
print("\n" + "="*60)
print(f"📊 Tamaño del conjunto después de corrección: {({"rows": clientes_Banco.count(), "cols": len(clientes_Banco.columns)})}")
print("="*60)

In [ ]:
print("="*60)
print("🎯 LIMPIEZA DE REGISTROS CON DURACIÓN NEGATIVA O NULA")
print("="*60)

# Mostrar tamaño inicial
print(f"\n📊 Tamaño del conjunto antes de corrección: {({"rows": clientes_Banco.count(), "cols": len(clientes_Banco.columns)})}")

# Contar registros con duración <= 0
duraciones_invalidas = clientes_Banco['duration'][clientes_Banco['duration'] <= 0].count()
print(f"🔍 Registros con duración ≤ 0 segundos: {duraciones_invalidas}")

# Mostrar ejemplos (solo si existen)
if duraciones_invalidas > 0:
    print("\n⚠️ Ejemplos de registros con duración inválida:")
    print(clientes_Banco.loc[clientes_Banco['duration'] <= 0].head())
else:
    print("\n✅ No se encontraron registros con duración ≤ 0.")

# Eliminar registros inválidos
clientes_Banco = clientes_Banco[clientes_Banco['duration'] > 0]

# Mostrar resultados finales
print("\n🧹 Registros con duración ≤ 0 eliminados correctamente.")
print(f"📊 Tamaño del conjunto después de corrección: {({"rows": clientes_Banco.count(), "cols": len(clientes_Banco.columns)})}")
print("="*60)


In [ ]:
print("="*60)
print("🎯 LIMPIEZA DE REGISTROS CON 'previous' MAYOR A 100")
print("="*60)

# Tamaño inicial
print(f"\n📊 Tamaño del conjunto antes de corrección: {({"rows": clientes_Banco.count(), "cols": len(clientes_Banco.columns)})}")

# Contar registros con previous > 100
previous_invalidos = clientes_Banco['previous'][clientes_Banco['previous'] > 100].count()
print(f"🔍 Registros con 'previous' > 100: {previous_invalidos}")

# Mostrar ejemplos si existen
if previous_invalidos > 0:
    print("\n⚠️ Ejemplos de registros con 'previous' fuera de rango:")
    print(clientes_Banco.loc[clientes_Banco['previous'] > 100].head())
else:
    print("\n✅ No se encontraron registros con 'previous' mayor a 100.")

# Eliminar registros inválidos
clientes_Banco = clientes_Banco[clientes_Banco['previous'] <= 100]

# Tamaño final
print("\n🧹 Registros con 'previous' > 100 eliminados correctamente.")
print(f"📊 Tamaño del conjunto después de corrección: {({"rows": clientes_Banco.count(), "cols": len(clientes_Banco.columns)})}")
print("="*60)


In [ ]:
# No se tienen columnas en este formato
# Pero para cambiar este tipo de datos se usa
#df = df.withColumn("fecha", pd.to_datetime(df["fecha"].dt.strftime('%Y-%m-%d')))


In [ ]:
# df=df.replace(to_replace=r'.\(.+\)$', value='', regex=True)
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

spark = SparkSession.builder.appName("BankExample").getOrCreate()
df = pd.DataFrame({'X': ["zeppy", "amid", "amily"],
                   'Y': ["xar", "abc", "among"],
                   'Z': ['11.2 (15:30)', '10.0 (03:40)', '64 (22:20)']})
df

In [ ]:
# En este caso no requiere dividir columnas y su combinacion se analizara con las variables
# categóricas se deja para que
df1 = df
Z = df1["Z"].str.split(expand=True)
Z.columns = ['Dato', 'Hora']
print(Z)
print('\n')
df1 = pd.concat([df1, Z], axis=1)
print(df1)
print('\n')
df1.drop(columns=['Z'], inplace=True)
df1

In [ ]:
print("Before Replacement")
print(df)
df.replace(to_replace=r'^ami.$', value='song', regex=True,inplace=True)
df

In [ ]:
print("After Replacement")
print(df)
print("After Replacement Z")
df=df.replace(to_replace=r'.\(.+\)$', value='', regex=True)
df

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

cols_cat = ['job', 'marital', 'education', 'default', 'housing',
            'loan', 'contact', 'month', 'poutcome', 'y']

sns.set_theme(style="whitegrid", palette="pastel")

fig, axes = plt.subplots(nrows=len(cols_cat), ncols=1, figsize=(12, 40))
fig.subplots_adjust(hspace=0.8)

for i, col in enumerate(cols_cat):
    sns.countplot(
        x=col,
        data=clientes_Banco,
        ax=axes[i],
        order=clientes_Banco[col].value_counts().index
    )
    axes[i].set_title(f"Distribución de la variable: {col}", fontsize=14, fontweight='bold')
    axes[i].set_xlabel("")
    axes[i].set_ylabel("Frecuencia", fontsize=12)
    axes[i].tick_params(axis='x', rotation=30)  # ✅ Forma correcta de rotar etiquetas
    axes[i].grid(True, linestyle='--', alpha=0.6)

fig.suptitle("Distribución de Variables Categóricas del Dataset", fontsize=18, fontweight='bold', color='darkblue')
plt.show()



In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Convertir las variables categóricas a minúsculas (seguras)
for column in clientes_Banco.columns:
    if column in cols_cat and clientes_Banco[column].dtype == 'object':
        clientes_Banco[column] = clientes_Banco[column].str.lower()

# Configuración de estilo
sns.set_theme(style="whitegrid", palette="pastel")

# Crear figura y ejes
fig, axes = plt.subplots(nrows=len(cols_cat), ncols=1, figsize=(12, 40))
fig.subplots_adjust(hspace=0.8)

# Graficar cada variable categórica
for i, col in enumerate(cols_cat):
    sns.countplot(
        x=col,
        data=clientes_Banco,
        ax=axes[i],
        order=clientes_Banco[col].value_counts().index
    )
    axes[i].set_title(f"Distribución de la variable: {col}", fontsize=14, fontweight='bold')
    axes[i].set_xlabel("")
    axes[i].set_ylabel("Frecuencia", fontsize=12)
    axes[i].tick_params(axis='x', rotation=30)  # ✅ Rotación sin warnings
    axes[i].grid(True, linestyle='--', alpha=0.6)

# Título global
fig.suptitle("Distribución de Variables Categóricas (en minúsculas)", fontsize=18, fontweight='bold', color='darkblue')
plt.show()


In [ ]:
# job: unificar admin. y administrative
print(clientes_Banco['job'].unique())
clientes_Banco = clientes_Banco.withColumn("job", clientes_Banco['job'].str.replace('admin.','administrative', regex=False))
print(clientes_Banco['job'].unique())

In [ ]:
# marital: unificar div. y divorced
print(clientes_Banco['marital'].unique())
clientes_Banco = clientes_Banco.withColumn("marital", clientes_Banco['marital'].str.replace('div.','divorced', regex=False))
print(clientes_Banco['marital'].unique())

In [ ]:
# education: unificar sec. y secondary, unk y unknown
print(clientes_Banco['education'].unique())
clientes_Banco = clientes_Banco.withColumn("education", clientes_Banco['education'].str.replace('sec.','secondary', regex=False))
clientes_Banco.loc[clientes_Banco = clientes_Banco.withColumn("education", ='unk','education'] = 'unknown')
print(clientes_Banco['education'].unique())

In [ ]:
# contact: unificar telephone y phone
print(clientes_Banco['contact'].unique())
clientes_Banco.loc[clientes_Banco = clientes_Banco.withColumn("contact", ='phone','contact'] = 'telephone')
clientes_Banco.loc[clientes_Banco = clientes_Banco.withColumn("contact", ='mobile','contact'] = 'cellular')
print(clientes_Banco['contact'].unique())

In [ ]:
# poutcome: unificar unk y unknown
print(clientes_Banco['poutcome'].unique())
clientes_Banco.loc[clientes_Banco = clientes_Banco.withColumn("poutcome", ='unk','poutcome']='unknown')
print(clientes_Banco['poutcome'].unique())

In [ ]:
print(({"rows": clientes_Banco.count(), "cols": len(clientes_Banco.columns)}))
clientes_Banco

In [ ]:
#exportar a csv
clientes_Banco.to_csv('clientes_Banco_clean.csv', index=False)


In [ ]:
# Notes / TODOs
# - If the original notebook used .corr() matrix or seaborn pairplots, consider:
#     numeric = [f.name for f in df.schema.fields if isinstance(f.dataType, (IntegerType, LongType, DoubleType, FloatType))]
#     # Example of pairwise correlations:
#     for i, c1 in enumerate(numeric):
#         for c2 in numeric[i+1:]:
#             print(c1, c2, df.stat.corr(c1, c2))
# - For plots, convert to pandas on small samples: df.sample(0.1).toPandas().plot(...)
